In [ ]:
#import package
import hashlib
from datetime import date, datetime
import random
import matplotlib.pylab as plt
import numpy as np
from tqdm import notebook
from tqdm.notebook import trange
import binascii
import struct
import csv
from numpy import linalg as LA
from numpy.linalg import matrix_rank
import random
from math import sqrt

In [ ]:
# Target Board Setup 
if False:
    SCOPETYPE = 'CWNANO'
    PLATFORM = 'CWNANO'
    SS_VER='SS_VER_1_1'
if False:  
    SCOPETYPE = 'CW308_STM32F3'
    PLATFORM = 'CW308_STM32F3'
    SS_VER='SS_VER_1_1'
if True:
    SCOPETYPE = 'OPENADC'
    PLATFORM = 'CWLITEXMEGA'
    SS_VER='SS_VER_1_1'

In [ ]:
%run /home/vagrant/work/projects/chipwhisperer/jupyter/Setup_Scripts/Setup_Generic.ipynb

In [ ]:
#Compile Program to Run on Board
PATH = "/home/vagrant/work/projects/chipwhisperer/hardware/victims/firmware/"
TARGET_MODEL = "C_NeuralNetwork" 

In [ ]:
%%bash -s "$PLATFORM" "$PATH" "$TARGET_MODEL" "$SS_VER"
cd $2$3 #change directory to our target model
make PLATFORM=$1 CRYPTO_TARGET=NONE SS_VER=$4

In [ ]:
#Programming the Board
if TARGET_MODEL == "C_NeuralNetwork":
    TAG = "NN"
    print('> Programming CW Lite with NeuralNetwork target:')
    print(f'>    {PATH+TARGET_MODEL}/NeuralNetwork-{PLATFORM}.hex')
    cw.program_target(scope, prog, f'{PATH+TARGET_MODEL}/NeuralNetwork-{PLATFORM}.hex')
else:
    print('BROKEN!!! UNIDENTIFIED TARGET MODEL')

In [ ]:
#Pass Model Data Over Serial
NUM_SAMPLES = 713 
f = open(f"{PATH+TARGET_MODEL}/pima-indians-diabetes.csv", "r")
lines = f.readlines()
print(f">> Sending {NUM_SAMPLES} samples over serial")
count = 0
for i in notebook.tnrange(NUM_SAMPLES, desc='Data Transfer'): 
    line = lines[i]
    # Build byte array
    line_list = [int(l) for l in line.replace('\n','').split(',')]
    ba = bytearray(line_list) 
    target.simpleserial_write('d',ba)
    print(f"  Original: {ba}")
    response = target.simpleserial_read('r',6,ack=False) # Verify response
    print(response)
expected_count = NUM_SAMPLES

In [ ]:
# Initialize Weight Matrices
ba = bytearray([1])
target.simpleserial_write('i', ba)
ret = scope.capture()
if ret: print("Target timed out!")
response = target.simpleserial_read('r', 2)
count = int.from_bytes(response, byteorder='little', signed=False)
expected_count = expected_count + 1
if(count != expected_count):
    print(f"Read coun does not match!!")
    print(f"  Expected: {NUM_SAMPLES+1}")
    print(f"    Actual: {count}")
else:
    print(">> Weights Initialized successfully:)")

In [ ]:
#Train Model and Collect Power Traces
scope.adc.samples = 5000 
num_epochs = 1200  
train_traces = []
plt.figure()
for i in notebook.tnrange(num_epochs, desc='Training Trace Capture'):     
    scope.arm() 
    b = bytearray([2]) 
    target.simpleserial_write('e', b)
    time.sleep(2)
    # Check for timeout
    ret = scope.capture()
    if ret: print("Target timed out!")
        
    response = target.simpleserial_read('r', 2, ack=False)
    count = int.from_bytes(response, byteorder='little', signed=False)
    expected_count = expected_count +1
    if(count != expected_count):
        print(f"Read count does not match!!")
        print(f"  Expected: {expected_count}")
        print(f"    Actual: {count}")
    train_traces.append(scope.get_last_trace())
    plt.plot(train_traces[i])
print(f">> Captured {len(train_traces[0])} traces for each of {len(train_traces)} training samples")
plt.show()

In [ ]:
#save power traces into csv
dt = datetime.strptime(str(datetime.now()).split('.')[0], '%Y-%m-%d %H:%M:%S')
timestamp = dt.timestamp() # assume dt is a local time
datehex = str(binascii.hexlify(struct.pack('<I', round(timestamp))))[1:].replace("'","")
filename_train = f'{TAG}_train_{datehex}.csv'
# Save Training Traces
f_train = open(filename_train, 'w')
csvwriter = csv.writer(f_train)
csvwriter.writerows(train_traces)
print(f">> Traces written to {filename_train}")